In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import wandb
import numpy as np

In [ ]:
if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
else:
        print("GPU is not available, using CPU.")
        
device = torch.device("cuda")

In [ ]:
os.environ["WANDB_API_KEY"] = "3aaf9f796df65417b3f5f8560b43875171b55805"

In [ ]:
# Generate a Sine Wave
steps = np.linspace(0, 100, 1000)
data = np.sin(steps)

# Create sequences: use 20 points to predict the 21st
seq_len = 20
X, Y = [], []
for i in range(len(data) - seq_len):
    X.append(data[i : i + seq_len])
    Y.append(data[i + seq_len])

X = torch.tensor(np.array(X), dtype=torch.float32).unsqueeze(-1) # (batch, seq, 1)
Y = torch.tensor(np.array(Y), dtype=torch.float32).unsqueeze(-1) # (batch, 1)

In [ ]:
print(X, Y)

In [ ]:
class SineRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(1, 32, batch_first=True)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        _, h = self.rnn(x)
        return self.fc(h[-1])

In [ ]:
def train_model(config=None):
    global X, Y
    # Initialize W&B
    wandb_run = wandb.init(
        project="VICI_test",
        entity="fabian-dubach-hochschule-luzern",
        config=config
    )
    config = wandb.config

    print("\n" + "=" * 60)
    print("Starting run with config:")
    for k, v in dict(config).items():
        print(f"  {k}: {v}")
    print("=" * 60 + "\n")
    
    # Setup Training
    model = SineRNN() # Using the SineRNN class defined previously
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    
    model = model.to(device)
    print("Model moved to GPU.")
    X = X.to(device)
    Y = Y.to(device)
    print("X and Y moved to GPU.")

    # Training Loop
    print("Starting Test Run...")
    for epoch in range(config['epochs']):
        optimizer.zero_grad()
        output = model(X) # Ensure X and Y are defined in your global scope
        loss = criterion(output, Y)
        loss.backward()
        optimizer.step()
        
        wandb.log({
            "epoch": epoch,
            "learning_rate": config['learning_rate'],
            "loss": loss.item()
        })
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")

    # --- SAVE CHECKPOINT ---
    checkpoint_path = "latest_model.pth"
    torch.save(model.state_dict(), checkpoint_path)
    print(f"\nModel saved to {checkpoint_path}")
    
    wandb.finish()


In [ ]:
# Run the training
train_config = {"learning_rate": 0.01, "epochs": 20}
train_model(config=train_config)

In [ ]:
# 1. Instantiate the architecture again
test_model = SineRNN()

# 2. Load the saved weights
test_model.load_state_dict(torch.load("latest_model.pth", weights_only=True))
test_model = test_model.to(device)
test_model.eval() # Set to evaluation mode

# 3. Run a quick test prediction
with torch.no_grad():
    sample_input = X[:1] # Take the first sequence from your data
    prediction = test_model(sample_input)
    
    print("Checkpoint Test:")
    print(f"Input Sequence Shape: {sample_input.shape}")
    print(f"Predicted Value: {prediction.item():.4f}")
    print(f"Actual Value:    {Y[0].item():.4f}")
    print("\nCheckpoint successfully loaded and verified!")